In [2]:
# get libs
import os
import json
import time
import openai
from dotenv import load_dotenv

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")
client = openai.OpenAI()

In [3]:
# function to print json
def show_json(obj):
    display(json.loads(obj.model_dump_json()))

In [4]:
# function to wait to get run
def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run

In [5]:
# create assistant
assistant = client.beta.assistants.create(
    name="Math Tutor",
    instructions="You are a personal math tutor. Answer questions briefly, in a sentence or less.",
    model="gpt-4-1106-preview",
)
show_json(assistant)

{'id': 'asst_ZXTgk6H2n8yzLwEZ2qRKz84C',
 'created_at': 1699926904,
 'description': None,
 'file_ids': [],
 'instructions': 'You are a personal math tutor. Answer questions briefly, in a sentence or less.',
 'metadata': {},
 'model': 'gpt-4-1106-preview',
 'name': 'Math Tutor',
 'object': 'assistant',
 'tools': []}

In [6]:
# create threads
thread = client.beta.threads.create()
show_json(thread)

{'id': 'thread_LqqkK2Ki2GIkOUYZUl8cCr0Z',
 'created_at': 1699926905,
 'metadata': {},
 'object': 'thread'}

In [7]:
# create message
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I need to solve the equation `3x + 11 = 14`. Can you help me?",
)
show_json(message)

{'id': 'msg_O0CDoErXIzTe1qLpPFKk5dZm',
 'assistant_id': None,
 'content': [{'text': {'annotations': [],
    'value': 'I need to solve the equation `3x + 11 = 14`. Can you help me?'},
   'type': 'text'}],
 'created_at': 1699926906,
 'file_ids': [],
 'metadata': {},
 'object': 'thread.message',
 'role': 'user',
 'run_id': None,
 'thread_id': 'thread_LqqkK2Ki2GIkOUYZUl8cCr0Z'}

In [8]:
# create run
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)
show_json(run)

{'id': 'run_npPtULjERwowyCcsW65apltE',
 'assistant_id': 'asst_ZXTgk6H2n8yzLwEZ2qRKz84C',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1699926906,
 'expires_at': 1699927506,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'You are a personal math tutor. Answer questions briefly, in a sentence or less.',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-4-1106-preview',
 'object': 'thread.run',
 'required_action': None,
 'started_at': None,
 'status': 'queued',
 'thread_id': 'thread_LqqkK2Ki2GIkOUYZUl8cCr0Z',
 'tools': []}

In [9]:
# wait on run
run = wait_on_run(run, thread)
show_json(run)

{'id': 'run_npPtULjERwowyCcsW65apltE',
 'assistant_id': 'asst_ZXTgk6H2n8yzLwEZ2qRKz84C',
 'cancelled_at': None,
 'completed_at': 1699926908,
 'created_at': 1699926906,
 'expires_at': None,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'You are a personal math tutor. Answer questions briefly, in a sentence or less.',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-4-1106-preview',
 'object': 'thread.run',
 'required_action': None,
 'started_at': 1699926906,
 'status': 'completed',
 'thread_id': 'thread_LqqkK2Ki2GIkOUYZUl8cCr0Z',
 'tools': []}

In [10]:
# check response
messages = client.beta.threads.messages.list(thread_id=thread.id)
show_json(messages)

{'data': [{'id': 'msg_ImXzQytnsrfpfBhd5bvfi7Ux',
   'assistant_id': 'asst_ZXTgk6H2n8yzLwEZ2qRKz84C',
   'content': [{'text': {'annotations': [],
      'value': 'Yes, to solve the equation `3x + 11 = 14`, first subtract 11 from both sides to get `3x = 3`, and then divide both sides by 3 to find `x = 1`.'},
     'type': 'text'}],
   'created_at': 1699926907,
   'file_ids': [],
   'metadata': {},
   'object': 'thread.message',
   'role': 'assistant',
   'run_id': 'run_npPtULjERwowyCcsW65apltE',
   'thread_id': 'thread_LqqkK2Ki2GIkOUYZUl8cCr0Z'},
  {'id': 'msg_O0CDoErXIzTe1qLpPFKk5dZm',
   'assistant_id': None,
   'content': [{'text': {'annotations': [],
      'value': 'I need to solve the equation `3x + 11 = 14`. Can you help me?'},
     'type': 'text'}],
   'created_at': 1699926906,
   'file_ids': [],
   'metadata': {},
   'object': 'thread.message',
   'role': 'user',
   'run_id': None,
   'thread_id': 'thread_LqqkK2Ki2GIkOUYZUl8cCr0Z'}],
 'object': 'list',
 'first_id': 'msg_ImXzQytnsrf